In [7]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv


In [12]:
load_dotenv()

DB_STRING = f"postgresql://{os.getenv('USER_DB')}:{os.getenv('PASSWORD')}@{os.getenv('HOST')}:{os.getenv('PORT')}/{os.getenv('DATABASE')}"
db = create_engine(DB_STRING)
#import the data to a pandas dataframe
query_string = """
    SELECT kchd.*, kchs.date, kchs.price
    FROM eda.king_county_house_details kchd
    LEFT JOIN eda.king_county_house_sales kchs
    ON kchd.id = kchs.house_id
"""
df_sqlalchemy = pd.read_sql(query_string, db)

df_sqlalchemy.head()
#export the data to a csv-file
df_sqlalchemy.to_csv('eda.csv',index=False)
#import the data from a csv-file
df_import = pd.read_csv('data/eda.csv')

In [40]:
conditions_df = df_import.query('bathrooms >=4 and waterfront == 0 and yr_built < 1975')
conditions_df.loc[:,'sqft_outside'] = conditions_df['sqft_lot'] - conditions_df['sqft_living'] - conditions_df['sqft_basement'].fillna(0) #or ignore the one row with na?
conditions_df


,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date,price,sqft_outside
779,1954700695,5.0,4.25,4860.0,9453.0,1.5,0.0,1.0,5,10,...,1905,0.0,98112,47.6196,-122.286,3150.0,8557.0,2014-06-12,2250000.0,2833.0
1985,1118002090,3.0,4.25,2820.0,7200.0,2.0,0.0,0.0,4,10,...,1930,0.0,98112,47.6298,-122.290,3300.0,7522.0,2014-06-28,1600000.0,4020.0
2038,9471200200,4.0,4.25,5040.0,16048.0,1.0,0.0,3.0,3,10,...,1950,NaN,98105,47.6702,-122.260,3960.0,14000.0,2015-03-25,2530000.0,9388.0
2650,7611200136,4.0,4.00,3770.0,9750.0,1.0,0.0,0.0,4,9,...,1967,NaN,98177,47.7159,-122.367,2260.0,9878.0,2014-07-23,872000.0,4150.0
3018,3377900195,4.0,5.50,6930.0,45100.0,1.0,0.0,0.0,4,11,...,1950,19910.0,98006,47.5547,-122.144,2560.0,37766.0,2014-09-29,2530000.0,38170.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18711,239000155,5.0,4.50,3540.0,21217.0,2.0,0.0,0.0,4,8,...,1926,0.0,98188,47.4274,-122.280,1290.0,12040.0,2015-01-05,707000.0,17077.0
19093,2025059131,4.0,4.25,3250.0,11780.0,2.0,0.0,0.0,3,8,...,1944,20010.0,98004,47.6322,-122.203,1800.0,9000.0,2014-09-04,980000.0,7640.0
19261,2397100705,4.0,4.25,3650.0,5328.0,1.5,0.0,0.0,3,9,...,1907,20140.0,98119,47.6380,-122.362,1710.0,3600.0,2014-07-14,1520000.0,358.0
19312,2771604190,7.0,4.25,3670.0,4000.0,2.0,0.0,1.0,3,8,...,1964,0.0,98199,47.6375,-122.388,2010.0,4000.0,2014-06-17,824000.0,-540.0


In [108]:
space_for_court_and_pool = 4800 + 800 #tennis court 30x15m recommended - about 4800, residential pool 800sqft, olympic pool 13.123sqft
df_outside = conditions_df[conditions_df['sqft_outside'] > space_for_court_and_pool + 400].copy() #buffer space outside
df_outside['sqft_outside_left_after_cond'] = df_outside['sqft_outside'] - (space_for_court_and_pool + 400)

#df_outside['sqft_outside_left_after_cond'].sort_values(ascending=False) #this will only get that one column
df_outside.sort_values(by= 'sqft_outside_left_after_cond', ascending=False)


,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date,price,sqft_outside,sqft_outside_left_after_cond
15152,3304700130,4.0,4.00,3860.0,67953.0,2.0,0.0,2.0,4,12,...,0.0,98177,47.7469,-122.378,4410.0,128066.0,2015-01-28,1760000.0,64093.0,58093.0
3314,2821049048,4.0,4.25,2360.0,57514.0,2.0,0.0,0.0,4,8,...,NaN,98003,47.2843,-122.294,2037.0,35733.0,2014-06-03,590000.0,55154.0,49154.0
14542,2303900035,5.0,6.25,8670.0,64033.0,2.0,0.0,4.0,3,13,...,NaN,98177,47.7295,-122.372,4140.0,81021.0,2014-06-11,2890000.0,52813.0,46813.0
3018,3377900195,4.0,5.50,6930.0,45100.0,1.0,0.0,0.0,4,11,...,19910.0,98006,47.5547,-122.144,2560.0,37766.0,2014-09-29,2530000.0,38170.0,32170.0
5961,5249800010,4.0,4.25,6410.0,43838.0,2.5,0.0,2.0,4,12,...,0.0,98144,47.5703,-122.280,2270.0,6630.0,2014-12-03,2730000.0,36628.0,30628.0
19445,722059020,6.0,4.50,4520.0,40164.0,2.0,0.0,0.0,3,9,...,20080.0,98031,47.4070,-122.216,2870.0,13068.0,2015-03-18,550000.0,34704.0,28704.0
4020,9175600025,7.0,6.75,7480.0,41664.0,2.0,0.0,2.0,3,11,...,NaN,98166,47.4643,-122.368,2810.0,33190.0,2014-10-07,800000.0,31784.0,25784.0
8507,3425059141,7.0,4.00,3150.0,34830.0,1.0,0.0,0.0,3,9,...,20050.0,98007,47.6029,-122.147,2390.0,12054.0,2014-05-28,999000.0,31680.0,25680.0
14172,1333300145,3.0,4.00,4200.0,30120.0,2.0,0.0,2.0,4,11,...,0.0,98112,47.6379,-122.311,2760.0,12200.0,2015-03-04,2230000.0,25320.0,19320.0
17665,3585901085,6.0,4.50,3810.0,28176.0,1.0,0.0,4.0,5,10,...,0.0,98177,47.7612,-122.381,3810.0,26400.0,2014-06-04,2010000.0,24366.0,18366.0


In [ ]:
#need to look at outliers before this


average_price = df_import['price'].mean()
q3_price = df_import['price'].quantile(0.75)

more_than_mean = df_outside['price'] > average_price
more_than_q3 = df_outside['price'] > q3_price
more_than_q3.sum()
(~more_than_q3).sum() #calculate how many False

print( f"not more than mean: {(~more_than_mean).sum()}. not in q3: {(~more_than_q3).sum()}")

not more than mean: 1. not in q3: 4


In [104]:
not_in_q3 = df_outside[df_outside['price'] < q3_price]
not_in_q3

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date,price,sqft_outside,sqft_outside_left_after_cond
3314,2821049048,4.0,4.25,2360.0,57514.0,2.0,0.0,0.0,4,8,...,NaN,98003,47.2843,-122.294,2037.0,35733.0,2014-06-03,590000.0,55154.0,49154.0
8608,5379801972,5.0,4.00,1400.0,8580.0,1.0,0.0,0.0,5,7,...,0.0,98188,47.4560,-122.292,1220.0,8832.0,2014-08-18,265000.0,6680.0,680.0
10264,1420700030,7.0,4.00,2690.0,10880.0,1.0,0.0,0.0,4,8,...,0.0,98033,47.6787,-122.168,1840.0,10836.0,2014-09-22,597157.0,8190.0,2190.0
19445,722059020,6.0,4.50,4520.0,40164.0,2.0,0.0,0.0,3,9,...,20080.0,98031,47.4070,-122.216,2870.0,13068.0,2015-03-18,550000.0,34704.0,28704.0


In [90]:
df_import.groupby('zipcode').count()['id']

zipcode
98001    361
98002    199
98003    280
98004    317
98005    168
        ... 
98177    255
98178    262
98188    136
98198    280
98199    317
Name: id, Length: 70, dtype: int64

In [107]:
df_import.bedrooms.unique()

array([ 3.,  2.,  4.,  5.,  1.,  6.,  7.,  8.,  9., 11., 10., 33.])